# Setup

In [8]:
!pip install 'crewai[tools]' -q
!pip install 'langchain-openai' -q

In [ ]:
# Set the API Keys as environment variables
from google.colab import userdata
openai_api_key = userdata.get('ai_agents_openai')
serper_api_key = userdata.get('serper_api')

import os
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['SERPER_API_KEY'] = serper_api_key

In [10]:
# Import libraries
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [ ]:
# Define the LLM
llm = ChatOpenAI(model_name='gpt-4', temperature=0.5)

# User Inputs

In [12]:
# Call for inputs: Interviwer, company, job position, job description:
interviewer = input("Enter the name of the interviewer (if unknown include None): ")
company = input("Enter the name of the company: ")
job_position = input("Enter the job position: ")
job_description = input("Enter the job description: ")

Enter the name of the interviewer (if unknown include None): Matthias Glöel
Enter the name of the company: Beat81
Enter the job position: Data Analytics Working Student
Enter the job description: About the Role As a Working Student in our Analytics Team, you will work closely with our Tech, Product, Marketing and Operations team to support our daily operations with analytical insights and help us to grow. You will be joining a dynamic, fast-paced start-up where you’ll have ownership and impact from day one!  Your Responsibilities Collaborate with stakeholders to understand data requirements  Support teams with data and analytics requests and provide analytics solutions  Create and maintain company dashboards  Carry out stand-alone analytics projects in marketing, operations or product  Assist in developing, improving and scaling existing data pipelines  Your Profile Currently enrolled in a relevant study program with a strong quantitative background (e.g. mathematics, computer science,

# Research Crew

## Research AI Agent

In [13]:
# Create research AI Agent
research_agent = Agent(
    role = "Research Agent",
    goal = "Conduct in-depth research",
    backstory = """ As a research specialist, your mission is to uncover detailed and relevant information"
    The ultimate goal is to meet the expectations of the task""",
    llm = llm,
    tools = [ScrapeWebsiteTool(), SerperDevTool()]
)

# Research Tasks

In [14]:
# Define the task about researching the company
research_company_task = Task(
    description = f"""
    Research about {company}, their position in the sector and recent industry trends.
    Focus on recent news and developments from last 12 months""",
    expected_output = f""" A comprehensive report in Markdown with bulled point about {company} and its industry""",
    agent = research_agent
)

In [16]:
# Research Interviewer Task
research_person_task = Task(
    description = f"Research about {interviewer} with a focus on their professional career",
    expected_output = f""" A comprehensive report in Markdown with bulled point about {interviewer} and its industry""",
    agent = research_agent
)

# AI Coach + Interview Prep

In [17]:
# Create the AI interview Coach
coach_agent = Agent(
    role = "Interview Coach",
    goal = f"I help the user prepare for the job interview for job {job_position}",
    backstory = f"You are an expert in job interviews in companies like {company}",
    llm = llm
)

In [18]:
# Prepare the list of questions
define_questions_tasks = Task(
    description = f"""
    Define a list of questions based on the company {company}, job description {job_description} and interviewer {interviewer}""",
    expected_output = f""" A list of 20 questions in Markdown format that covers: culture and team fit, job position fit, background and ways of working, growth mindset""",
    agent = coach_agent,
    context = [research_company_task, research_person_task]
)

## Assemble the Crew

In [19]:
# Assembling the crew
crew = Crew(
    agents = [research_agent, coach_agent],
    tasks = [research_company_task, research_person_task, define_questions_tasks],
    verbose = True,
    process = Process.sequential
)
result = crew.kickoff({"topic": "Write a list of question to prepare for the interview"})

╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: 56050a1f-384c-4c6f-b5d7-73e8f6c2864c                                    │
│  Tool Args:                                                                  │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 6de1c79f-01b0-419d-9d56-7f6c1436ddf2
    Status: Executing Task...╭────────────────────────────── 🤖 Agent Started ──────────────────────────────╮
│                                                                              │
│  Agent: Research

# Exporting

In [20]:
# Check the crew outcome
display(Markdown(result.tasks_output[2].raw))

Here is a list of 20 questions that covers culture and team fit, job position fit, background and ways of working, and growth mindset for the Data Analytics Working Student position at Beat81:

### Culture and Team Fit
1. How does Beat81 foster a supportive and community-driven work environment?
2. Can you describe the team rituals and events that help build a strong team culture at Beat81?
3. How does the company ensure a good work-life balance for its employees, especially in a fast-paced start-up environment?
4. What qualities do you value most in a team member at Beat81?
5. How does Beat81 incorporate its fitness and wellness ethos into the workplace culture?
6. Can you share an example of how the team collaborates across different departments, such as Tech, Product, Marketing, and Operations?

### Job Position Fit
7. What are the key challenges that the Analytics Team is currently facing, and how can a Working Student contribute to overcoming these challenges?
8. How does the role of a Working Student in Analytics support Beat81’s growth and innovation strategies?
9. Can you provide examples of past projects that Working Students have been involved in within the Analytics Team?
10. What are the expectations for a Working Student's contribution to the creation and maintenance of company dashboards?
11. How does Beat81 measure the success of analytics solutions provided by the team?

### Background and Ways of Working
12. What specific skills and experiences are you looking for in a candidate for this role?
13. How does the team at Beat81 utilize tools like Google Sheets, SQL, and Python in their daily operations?
14. What is the typical workflow or process for handling data and analytics requests from other teams?
15. How does Beat81 support the professional development of its Working Students in terms of gaining new skills or industry knowledge?
16. Can you describe the type of mentorship or guidance a Working Student would receive from more experienced team members?

### Growth Mindset
17. How does Beat81 encourage continuous learning and growth among its employees, particularly in the Analytics Team?
18. What opportunities are available for a Working Student to take on more responsibilities or lead projects?
19. How does the company stay ahead of industry trends in fitness technology and data analytics?
20. Can you share an example of how a team member at Beat81 demonstrated innovation or took initiative to improve a process or solution?

These questions are designed to help the candidate understand the role, the company culture, and how they can contribute to and grow within Beat81.

In [21]:
# Export each of the tasks output
# Exporting the company report
file_name = f"{company}_report.txt"

# Open the file in write mode and write the content
with open(file_name, 'w') as file:
    file.write(result.tasks_output[0].raw)

# Export the interviewer report
file_name = f"{interviewer}_report.txt"

# Open the file in write mode and write the content
with open(file_name, 'w') as file:
    file.write(result.tasks_output[1].raw)

# Export the interview questions

file_name = f"{job_position}_questions.txt"

# Open the file in write mode and write the content
with open(file_name, 'w') as file:
    file.write(result.tasks_output[2].raw)

# Interview Prep

In [22]:
# Import the questions from the directory
with open(f"{job_position}_questions.txt", "r") as file:
  questions = file.read()

In [23]:
# AI Agent that asks a question
interviewer_agent = Agent(
    role = "Interviewer Agent",
    goal = f"Ask a question from {questions} to the user",
    backstory = f"You are an expert in job interviews in companies like {company}",
    llm = llm
)

In [27]:
# Interview Task
interview_prep_task = Task(
    description = f"Ask a question from {questions} to the user",
    expected_output = f"Ask only the question at a time",
    agent = interviewer_agent,
    human_input = True
)

In [30]:
# Feedback Task
feedback_task = Task(
    description = "Give feedback to the user on their answer",
    expected_output = "Markdown answer of what was good, what could be improved and how to take it to the next level",
    agent = coach_agent,
    context = [interview_prep_task],
    human_input = True
)

## Interview Prep Crew

In [31]:
# Assemble the interview Prep Crew
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, feedback_task],
    verbose = True,
    process = Process.sequential
)
result = crew.kickoff({"topic": f"Ask me all questions from {questions}"})

╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: 273ed32b-bf96-46f4-8886-e47fbb20fdb5                                    │
│  Tool Args:                                                                  │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

[CrewAIEventsBus] Sync handler error in on_task_started: Only one live display 
may be active at once
╭────────────────────────────── 🤖 Agent Started ──────────────────────────────╮
│                                                                              │
│  Agen

In [ ]:
# Assemble the interview Prep Crew
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, feedback_task],
    verbose = True,
    process = Process.hierarchical,
    manager_llm = ChatOpenAI(model_name = 'gpt-4', temperature = 0),
    memory = True,
    planning = True
)
result = crew.kickoff({"topic": f"Ask me all questions from {questions}"})

╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: 6d89b2ff-0575-412d-a926-7749c009090a                                    │
│  Tool Args:                                                                  │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯


[2025-10-24 16:54:18][INFO]: Planning the crew execution
[CrewAIEventsBus] Sync handler error in on_task_started: Only one live display 
may be active at once
╭────────────────────────────── Task Completion ───────────────────────────────╮
│                            

KeyboardInterrupt: Interrupted by user